In [83]:
import pandas as pd
from sodapy import Socrata
import datetime as dt
import matplotlib.pyplot as plt
import altair as alt

In [74]:
client = Socrata("data.seattle.gov", None)

In [75]:
client.timeout=180

In [76]:
query = """
SELECT
  usageclass,
  materialtype, 
  checkoutmonth, 
  checkoutyear, 
  sum(checkouts) as monthly_checkouts
WHERE
  checkoutyear >= 2015 
  and materialtype in ('EBOOK', 'BOOK', 'AUDIOBOOK')
GROUP BY
  usageclass, materialtype, checkoutmonth, checkoutyear
"""

In [77]:
result = client.get('tmmm-ytt6', query=query)

In [78]:
df = pd.DataFrame.from_records(result)

In [57]:
df.sort_values(by='monthly_checkouts', ascending=False)

,usageclass,materialtype,checkoutmonth,checkoutyear,monthly_checkouts
66,Digital,EBOOK,2,2015,99102
195,Digital,AUDIOBOOK,4,2019,99034
120,Digital,AUDIOBOOK,3,2019,97901
72,Digital,AUDIOBOOK,8,2018,90293
178,Digital,AUDIOBOOK,10,2018,90042
...,...,...,...,...,...
187,Digital,EBOOK,5,2015,106866
50,Digital,AUDIOBOOK,5,2019,106079
156,Digital,AUDIOBOOK,6,2019,105903
38,Digital,EBOOK,4,2015,105258


In [79]:
df['usageclass'] = df['usageclass'].astype('str')
df['materialtype'] = df['materialtype'].astype('str')
df['checkoutmonth'] = df['checkoutmonth'].astype('int')
df['checkoutyear'] = df['checkoutyear'].astype('int')
df['monthly_checkouts'] = df['monthly_checkouts'].astype('int')

In [80]:
df['date'] = df.apply(lambda x: dt.datetime(year=x['checkoutyear'], month=x['checkoutmonth'], day=1), axis=1)

In [81]:
df

,usageclass,materialtype,checkoutmonth,checkoutyear,monthly_checkouts,date
0,Physical,BOOK,7,2019,356688,2019-07-01
1,Physical,BOOK,3,2018,344487,2018-03-01
2,Physical,BOOK,9,2017,331058,2017-09-01
3,Physical,BOOK,10,2018,338497,2018-10-01
4,Physical,BOOK,11,2017,335034,2017-11-01
...,...,...,...,...,...,...
193,Digital,AUDIOBOOK,2,2018,68119,2018-02-01
194,Digital,EBOOK,5,2019,172007,2019-05-01
195,Digital,AUDIOBOOK,4,2019,99034,2019-04-01
196,Digital,EBOOK,11,2019,178939,2019-11-01


In [90]:
book_df = df[df['materialtype'].isin(['BOOK', 'EBOOK'])]

In [91]:
alt.Chart(book_df).mark_line().encode(
    x='date',
    y='monthly_checkouts',
    color='usageclass'
)

alt.Chart(...)